In [1]:
import pandas as pd


In [2]:
data = pd.read_csv('gds5393.csv',sep=',', index_col = 0)
meta = pd.read_csv('meta-gds5393.csv',sep=',', index_col = 0)
print(data.shape)
print(meta.shape)

(48107, 120)
(120, 6)


In [3]:
data4 = pd.read_csv('gds1686.csv',sep=',', index_col = 0)
meta4 = pd.read_csv('meta-gds1686.csv',sep=',', index_col = 0)
print(data4.shape)
print(meta4.shape)

(14010, 30)
(30, 4)


In [4]:
data6 = pd.read_csv('gds5473.csv',sep=',', index_col = 0)
meta6 = pd.read_csv('meta-gds5473.csv',sep=',', index_col = 0)
print(data6.shape)
print(meta6.shape)

(48107, 48)
(48, 4)
